In [57]:
import pandas as pd
import numpy as np
import random

In [58]:
# Prepare the data
customers_df = pd.read_csv("customers.csv")
vehicles_df = pd.read_csv("vehicles.csv")

In [59]:
customers_df.head()

,customer,latitude,longitude,demand
0,1,4.3555,113.9777,5
1,2,4.3976,114.0049,8
2,3,4.3163,114.0764,3
3,4,4.3184,113.9932,6
4,5,4.4024,113.9896,5


In [60]:
vehicles_df.head()

,vehicle,capacity,cost_per_km
0,Type A,25,1.2
1,Type B,30,1.5


In [61]:
# Convert the DataFrames to lists of dictionaries
customers = customers_df.to_dict('records')
vehicles_data = vehicles_df.to_dict('records')

print("Customers:")
print(customers)

print("\nVehicles:")
print(vehicles_data)

Customers:
[{'customer': 1, 'latitude': 4.3555, 'longitude': 113.9777, 'demand': 5}, {'customer': 2, 'latitude': 4.3976, 'longitude': 114.0049, 'demand': 8}, {'customer': 3, 'latitude': 4.3163, 'longitude': 114.0764, 'demand': 3}, {'customer': 4, 'latitude': 4.3184, 'longitude': 113.9932, 'demand': 6}, {'customer': 5, 'latitude': 4.4024, 'longitude': 113.9896, 'demand': 5}, {'customer': 6, 'latitude': 4.4142, 'longitude': 114.0127, 'demand': 8}, {'customer': 7, 'latitude': 4.0804, 'longitude': 114.0734, 'demand': 3}, {'customer': 8, 'latitude': 4.3818, 'longitude': 114.2034, 'demand': 6}, {'customer': 9, 'latitude': 4.4935, 'longitude': 114.1828, 'demand': 5}, {'customer': 10, 'latitude': 4.4932, 'longitude': 114.1322, 'demand': 8}]

Vehicles:
[{'vehicle': 'Type A', 'capacity': 25, 'cost_per_km': 1.2}, {'vehicle': 'Type B', 'capacity': 30, 'cost_per_km': 1.5}]


In [62]:
def calculate_distance(point1, point2):
    """
    Calculate the Euclidean distance between two geographical points.

    Args:
    point1: Tuple representing the latitude and longitude of the first point.
    point2: Tuple representing the latitude and longitude of the second point.

    Returns:
    Distance in kilometers between the two points.
    """
    return 100 * np.sqrt((point2[1] - point1[1])**2 + (point2[0] - point1[0])**2)

In [63]:
def generate_initial_population(pop_size, customers, vehicles_data):
    """
    Generate an initial population of solutions for the genetic algorithm.

    Args:
    pop_size: The number of solutions in the population.
    customers: A list of dictionaries containing customer data.
    vehicles_data: A list of dictionaries containing vehicle data.

    Returns:
    A list of solutions, where each solution is a list of routes for the vehicles.
    """
    population = []
    for _ in range(pop_size):
        solution = []
        remaining_customers = customers.copy()
        random.shuffle(remaining_customers)  

        while remaining_customers:
            for vehicle in vehicles_data:
                route = []
                route_demand = 0
                for customer in remaining_customers[:]:
                    if route_demand + customer['demand'] <= vehicle['capacity']:
                        route.append(customer)
                        route_demand += customer['demand']
                        remaining_customers.remove(customer)
                if route:
                    solution.append({'Vehicle': vehicle['vehicle'], 'route': route})
        
        population.append(solution)
    return population

In [64]:
def evaluate(solution):
    """
    Evaluate the fitness of a solution.

    Args:
    solution: A solution representing a set of routes for the vehicles.

    Returns:
    A float representing the total cost of the solution. If the solution is infeasible, returns a high penalty value.
    """
    total_distance = 0
    total_cost = 0
    
    for vehicle in solution:
        vehicle_type = vehicle['Vehicle']
        vehicle_info = next(item for item in vehicles_data if item['vehicle'] == vehicle_type)
        vehicle_capacity = vehicle_info['capacity']
        cost_per_km = vehicle_info['cost_per_km']
        
        route = vehicle['route']
        route_demand = sum(customer['demand'] for customer in route)
        
        if route_demand > vehicle_capacity:
            return float('inf')  
        
        route_distance = 0
        last_location = depot
        for customer in route:
            customer_location = (customer['latitude'], customer['longitude'])
            route_distance += calculate_distance(last_location, customer_location)
            last_location = customer_location
        route_distance += calculate_distance(last_location, depot)  
        total_distance += route_distance
        total_cost += route_distance * cost_per_km
    
    return total_cost


In [65]:
def selection(population, fitnesses):
    """
    Select individuals from the population based on their fitness.

    Args:
    population: A list of solutions.
    fitnesses: A list of fitness values corresponding to the solutions.

    Returns:
    A list of selected solutions for the next generation.
    """
    total_fitness = sum(1 / f for f in fitnesses)  
    selection_probs = [(1 / f) / total_fitness for f in fitnesses]
    
    selected = random.choices(population, weights=selection_probs, k=len(population))
    return selected

In [66]:
def crossover(parent1, parent2):
    """
    Perform crossover between two parent solutions to produce offspring.

    Args:
    parent1: The first parent solution.
    parent2: The second parent solution.

    Returns:
    Two offspring solutions created by combining the parents.
    """
    def get_vehicle_routes(solution):
        routes = []
        for vehicle in solution:
            routes.extend(vehicle['route'])
        return routes

    def create_offspring(parent1, parent2):
        route1 = get_vehicle_routes(parent1)
        route2 = get_vehicle_routes(parent2)
        
        size = len(route1)
        start, end = sorted(random.sample(range(size), 2))
        
        child_route = [None] * size
        child_route[start:end] = route1[start:end]
        
        pointer = 0
        for customer in route2:
            if customer not in child_route:
                while pointer < size and child_route[pointer] is not None:
                    pointer += 1
                if pointer < size:
                    child_route[pointer] = customer
        
        return child_route

    def distribute_customers(route, vehicles_data):
        solution = []
        remaining_customers = [c for c in route if c is not None]
        
        while remaining_customers:
            for vehicle in vehicles_data:
                route_segment = []
                route_demand = 0
                for customer in remaining_customers[:]:
                    if route_demand + customer['demand'] <= vehicle['capacity']:
                        route_segment.append(customer)
                        route_demand += customer['demand']
                        remaining_customers.remove(customer)
                if route_segment:
                    solution.append({'Vehicle': vehicle['vehicle'], 'route': route_segment})

        return solution
    
    child1_route = create_offspring(parent1, parent2)
    child2_route = create_offspring(parent2, parent1)
    
    child1 = distribute_customers(child1_route, vehicles_data)
    child2 = distribute_customers(child2_route, vehicles_data)
    
    return child1, child2

In [67]:
def mutate(solution, mutation_rate=0.1):
    """
    Mutate a solution by swapping two customers in a route.

    Args:
    solution: The solution to mutate.
    mutation_rate: The probability of mutation for each customer in a route.

    Returns:
    The mutated solution.
    """
    for vehicle in solution:
        route = vehicle['route']
        if random.random() < mutation_rate:
            if len(route) > 1:
                i, j = random.sample(range(len(route)), 2)
                route[i], route[j] = route[j], route[i]
    
    return solution

In [68]:
def genetic_algorithm(customers, vehicles_data, pop_size=100, generations=500, mutation_rate=0.1):
    """
    Run the Genetic Algorithm to solve the Vehicle Routing Problem.

    Args:
    customers: A list of dictionaries containing customer data.
    vehicles_data: A list of dictionaries containing vehicle data.
    depot: Tuple representing the coordinates of the depot.
    pop_size: The number of solutions in the population.
    generations: The number of generations to run the algorithm.
    mutation_rate: The probability of mutation for each customer in a route.

    Returns:
    The best solution found and its fitness value.
    """
    population = generate_initial_population(pop_size, customers, vehicles_data)
    
    for _ in range(generations):
        fitnesses = [evaluate(solution) for solution in population]
        
        selected_population = selection(population, fitnesses)
        
        new_population = []
        for i in range(0, len(selected_population), 2):
            parent1 = selected_population[i]
            parent2 = selected_population[i+1] if i+1 < len(selected_population) else selected_population[0]
            child1, child2 = crossover(parent1, parent2)
            new_population.append(mutate(child1, mutation_rate))
            new_population.append(mutate(child2, mutation_rate))
        
        population = new_population
    
    best_solution = min(population, key=evaluate)
    return best_solution

In [69]:
def format_output(best_solution):
    """
    Format the output to match the required sample output format.
    
    Args:
    best_solution: The best solution found by the genetic algorithm.
    
    Returns:
    A formatted string representing the solution and its details.
    """
    type_a_vehicles = []
    type_b_vehicles = []
    total_distance = 0
    total_cost = 0

    for i, vehicle in enumerate(best_solution):
        vehicle_type = vehicle['Vehicle']
        route = vehicle['route']
        route_demand = sum(customer['demand'] for customer in route)
        
        route_distance = 0
        last_location = depot
        route_details = []
        for customer in route:
            customer_location = (customer['latitude'], customer['longitude'])
            distance = calculate_distance(last_location, customer_location)
            route_distance += distance
            route_details.append(f" -> C{customer['customer']} ({distance:.3f} km)")
            last_location = customer_location
        route_distance += calculate_distance(last_location, depot)
        total_distance += route_distance
        
        vehicle_info = next(item for item in vehicles_data if item['vehicle'] == vehicle_type)
        cost_per_km = vehicle_info['cost_per_km']
        route_cost = route_distance * cost_per_km
        total_cost += route_cost
        
        route_string = f"Depot{''.join(route_details)} -> Depot ({calculate_distance(last_location, depot):.3f} km)"
        vehicle_output = f"Vehicle {i + 1} ({vehicle_type}):\n"
        vehicle_output += f"Round Trip Distance: {route_distance:.3f} km, Cost: RM {route_cost:.2f}, Demand: {route_demand}\n"
        vehicle_output += f"{route_string}\n"

        if vehicle_type == 'Type A':
            type_a_vehicles.append(vehicle_output)
        else:
            type_b_vehicles.append(vehicle_output)
    
    total_output = f"Total Distance = {total_distance:.3f} km\nTotal Cost = RM {total_cost:.2f}\n\n"
    return total_output + ''.join(type_a_vehicles) + ''.join(type_b_vehicles)

In [70]:
if __name__ == "__main__":
    depot = (4.4184, 114.0932)

    best_solution = genetic_algorithm(customers, vehicles_data)
    formatted_output = format_output(best_solution)
    print(formatted_output)

Total Distance = 156.844 km
Total Cost = RM 204.73

Vehicle 1 (Type A):
Round Trip Distance: 84.924 km, Cost: RM 101.91, Demand: 25
Depot -> C6 (8.061 km) -> C4 (9.776 km) -> C1 (4.021 km) -> C7 (29.127 km) -> C3 (23.592 km) -> Depot (10.347 km)
Vehicle 3 (Type A):
Round Trip Distance: 16.871 km, Cost: RM 20.25, Demand: 8
Depot -> C10 (8.436 km) -> Depot (8.436 km)
Vehicle 2 (Type B):
Round Trip Distance: 55.049 km, Cost: RM 82.57, Demand: 24
Depot -> C9 (11.691 km) -> C8 (11.358 km) -> C2 (19.913 km) -> C5 (1.604 km) -> Depot (10.483 km)

